In [2]:
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
import json

_ = load_dotenv(find_dotenv())

client = OpenAI()

## 1. Create an Assistant

In [3]:
assistant = client.beta.assistants.create(
    name="Javascrit Teacher",
    instructions="You are Javascript teacher. Write and run code to answer queries",
    tools=[{"type":"code_interpreter"}],
    model="gpt-3.5-turbo-1106",
)

dict(assistant)

{'id': 'asst_4oJAEqrRMp9CcqJaLC4bNbTw',
 'created_at': 1703524950,
 'description': None,
 'file_ids': [],
 'instructions': 'You are Javascript teacher. Write and run code to answer queries',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Javascrit Teacher',
 'object': 'assistant',
 'tools': [ToolCodeInterpreter(type='code_interpreter')]}

## 2. Create a Thread

In [4]:
thread = client.beta.threads.create()

print(thread)

thread_message = client.beta.threads.messages.list(thread.id)
dict(thread_message)  # there is no message right now

Thread(id='thread_d5gvYoSJtfsSjWMnyheLgzYo', created_at=1703524950, metadata={}, object='thread')


{}

## 3. Add a Message to a Thread

In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What are the primitive data types?"
)

dict(message)

{'id': 'msg_VQieOiXFFJnYn9KIOJLuhh9Y',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='What are the primitive data types?'), type='text')],
 'created_at': 1703524951,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_d5gvYoSJtfsSjWMnyheLgzYo'}

### List the Message in a Thread

In [10]:
thread_message = client.beta.threads.messages.list(thread.id)

print(thread_message) # now message has been appended to thread


SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_VQieOiXFFJnYn9KIOJLuhh9Y', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What are the primitive data types?'), type='text')], created_at=1703524951, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_d5gvYoSJtfsSjWMnyheLgzYo')], object='list', first_id='msg_VQieOiXFFJnYn9KIOJLuhh9Y', last_id='msg_VQieOiXFFJnYn9KIOJLuhh9Y', has_more=False)


## 4. Run the Assistant

In [11]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    # instructions="This student concepts are weak"
        # this is optional, it overrides the Assistant's default instruction
)

dict(run)

{'id': 'run_KhU93Prv2hAdH29jToPBkDru',
 'assistant_id': 'asst_4oJAEqrRMp9CcqJaLC4bNbTw',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1703525030,
 'expires_at': 1703525630,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are Javascript teacher. Write and run code to answer queries',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_d5gvYoSJtfsSjWMnyheLgzYo',
 'tools': [ToolAssistantToolsCode(type='code_interpreter')]}

## 5. Check the Run Assistant

In [12]:
# by default Run goes into 'queued' status

run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)

dict(run)

{'id': 'run_KhU93Prv2hAdH29jToPBkDru',
 'assistant_id': 'asst_4oJAEqrRMp9CcqJaLC4bNbTw',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1703525030,
 'expires_at': 1703525630,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are Javascript teacher. Write and run code to answer queries',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1703525031,
 'status': 'in_progress',
 'thread_id': 'thread_d5gvYoSJtfsSjWMnyheLgzYo',
 'tools': [ToolAssistantToolsCode(type='code_interpreter')]}

## 6. Display the Assistant's Response

In [13]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

print(messages)
print(messages.data)

print("")
# displaying in conversation style
for message in reversed(messages.data):
  # revesed() because assistant message was appended and it is listed first.
  print(message.role + ": " + message.content[0].text.value)

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_VQieOiXFFJnYn9KIOJLuhh9Y', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What are the primitive data types?'), type='text')], created_at=1703524951, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_d5gvYoSJtfsSjWMnyheLgzYo')], object='list', first_id='msg_VQieOiXFFJnYn9KIOJLuhh9Y', last_id='msg_VQieOiXFFJnYn9KIOJLuhh9Y', has_more=False)
[ThreadMessage(id='msg_VQieOiXFFJnYn9KIOJLuhh9Y', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What are the primitive data types?'), type='text')], created_at=1703524951, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_d5gvYoSJtfsSjWMnyheLgzYo')]

user: What are the primitive data types?
